Nhập thư viện và đọc dữ liệu từ file

In [ ]:
import pandas as pd
import seaborn as sns
import dython
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

df = pd.read_csv("data.csv",sep=";")

## 1. Khám phá dữ liệu (Exploratory Data Analysis - EDA)

### a. Tóm tắt thông tin dữ liệu:

In [ ]:
df.describe()

In [ ]:
df.head(5)

cái df này nó chỉ có cột target là cat thôi vì trên uci nó đã label encode r lọc ra thành mấy cột integer r nên h đi tìm lại để tìm tương quan

In [ ]:
cat_cols = df.select_dtypes(include=["object"])
print(cat_cols.columns)

In [ ]:
num_cols = df.select_dtypes(include=['number'])
print(num_cols.columns)

In [ ]:
cat_like = [col for col in df.columns if df[col].nunique() < 50]
print(cat_like)

In [ ]:
quantiative_columns = [
    'Age at enrollment',
    'Curricular units 1st sem (credited)',
    'Curricular units 1st sem (enrolled)',
    'Curricular units 1st sem (evaluations)',
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (without evaluations)',
    'Curricular units 2nd sem (credited)',
    'Curricular units 2nd sem (enrolled)',
    'Curricular units 2nd sem (evaluations)',
    'Curricular units 2nd sem (approved)',
    'Curricular units 2nd sem (without evaluations)',
    'Unemployment rate',
    'Inflation rate',
    'GDP'
]

cat_columns = [col for col in cat_like if col not in quantiative_columns]
ordinal_column = ["Application order"]
print(cat_columns)

#### Kiểm tra số lượng bản ghi, số lượng biến số, kiểu dữ liệu của từng biến.


In [ ]:
print(df.dtypes)


In [ ]:
df.shape

#### Xử lý dữ liệu thiếu, dữ liệu trùng lặp (nếu có).


In [ ]:
print(df.isna().sum())
print(df.isnull().sum())

### b. Phân tích thống kê mô tả

#### Tính các thống kê như trung bình, trung vị, độ lệch chuẩn, tứ phân vị.


In [ ]:
df_numeric = df.drop(columns=cat_columns)
df_numeric.describe()


In [ ]:
plt.figure(figsize=(10,6))

# Histogram split by target
sns.histplot(
    data=df,
    x='Admission grade',
    hue='Target',        # separates Dropout vs Graduate
    bins=20,             # adjust for clarity
    kde=True,            # optional: overlay density
    stat='density',      # normalize for comparison
    palette=['red', 'green','blue'],
    alpha=0.5
)

plt.title('Admission grade distribution by Target')
plt.xlabel('Admission grade')
plt.ylabel('Density')
plt.show()


#### Trực quan hoá bằng các biểu đồ phù hợp (ít nhất 3 loại biểu đồ) để nhận

In [ ]:
plt.figure(figsize=(10, 6))

# Create a countplot:
# x-axis: The outcome (Target)
# hue: The secondary variable used to group the bars (Attendance Status)
# palette: Uses the same color scheme as the previous example (reds/greens) for consistency
sns.countplot(
    data=df,
    x='Target',
    hue='Daytime/evening attendance\t',
    palette={'green','red'},
    edgecolor='black',
    linewidth=1.5
)

# --- 3. CUSTOMIZE PLOT AESTHETICS ---
plt.title('Student Outcome (Target) by Attendance Status', fontsize=16, fontweight='bold')
plt.xlabel('Student Outcome', fontsize=14)
plt.ylabel('Count of Students', fontsize=14)
plt.legend(title='Attendance Status', title_fontsize='12', fontsize='10')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Assuming df is your DataFrame
attendance_col = 'Daytime/evening attendance\t'  # keep the tab if it's in the column name
target_col = 'Target'

# Count occurrences
counts = df.groupby([attendance_col, target_col]).size().reset_index(name='count')

# Plot
plt.figure(figsize=(8,5))
sns.barplot(data=counts, x=attendance_col, y='count', hue=target_col)
plt.xlabel('Attendance Type (0=Daytime, 1=Evening)')
plt.ylabel('Number of Students')
plt.title('Attendance Type vs Final Target')
plt.legend(title='Target')
plt.show()


this data makes sense specially for this dataset since this is A dataset created from a higher education institution (acquired from several disjoint databases) related to students enrolled in different undergraduate degrees, such as agronomy, design, education, nursing, journalism, management, social service, and technologies. 

i.e. they have jobs in the day


In [ ]:
attendance_col = 'Daytime/evening attendance\t'
target_col = 'Target'

categories = ['Dropout', 'Enrolled', 'Graduate']

daytime_counts = df[df[attendance_col] == 0][target_col].value_counts().reindex(categories)
evening_counts = df[df[attendance_col] == 1][target_col].value_counts().reindex(categories)

# Create two side-by-side pie charts
fig, axes = plt.subplots(1, 2, figsize=(12,6))

axes[0].pie(daytime_counts, labels=daytime_counts.index, autopct='%1.1f%%', startangle=90, colors=["#ff0000",'#8da0cb','#66c2a5'])
axes[0].set_title('Daytime Attendance')

axes[1].pie(evening_counts, labels=evening_counts.index, autopct='%1.1f%%', startangle=90, colors=['#ff0000','#8da0cb','#66c2a5'])
axes[1].set_title('Evening Attendance')

plt.suptitle('Target Distribution by Attendance Type')
plt.show()


this one shows percantage

Same thing but for scholarship (this should be obvious)

In [ ]:
var_col = 'Scholarship holder'  # keep the tab if it's in the column name
target_col = 'Target'

# Count occurrences
counts = df.groupby([var_col, target_col]).size().reset_index(name='count')

# Plot
plt.figure(figsize=(8,5))
sns.barplot(data=counts, x=var_col, y='count', hue=target_col)
plt.xlabel('Scholarship holder (0=No, 1=Yes)')
plt.ylabel('Number of Students')
plt.title('Scholarship Type vs Final Target')
plt.legend(title='Target')
plt.show()

In [ ]:
var_col = 'Scholarship holder'  
target_col = 'Target'

categories = ['Dropout', 'Enrolled', 'Graduate']

no = df[df[var_col] == 0][target_col].value_counts().reindex(categories)
yes = df[df[var_col] == 1][target_col].value_counts().reindex(categories)

# Create two side-by-side pie charts
fig, axes = plt.subplots(1, 2, figsize=(12,6))

axes[0].pie(no, labels=no.index, autopct='%1.1f%%', startangle=90, colors=["#ff0000",'#8da0cb','#66c2a5'])
axes[0].set_title('Non scholarship holders')

axes[1].pie(yes, labels=yes.index, autopct='%1.1f%%', startangle=90, colors=['#ff0000','#8da0cb','#66c2a5'])
axes[1].set_title('Scholarship holders')

plt.suptitle('Target Distribution by Scholarship Type')
plt.show()


### c. Phát hiện dữ liệu ngoại lai

Sử dụng IQR (Interquartile Range)

In [ ]:
plt.figure(figsize=(15,8))
df_numeric.boxplot(rot=90)  # rot=90 to rotate x-axis labels
plt.title("Boxplots of All Numeric Columns")
plt.show()

**investigating gdp, inflation and unemployment**


In [ ]:
new_df = df[['GDP', 'Unemployment rate', 'Inflation rate']]
new_df.describe()

Knowing that the data comes from the Instituto Politécnico de Portalegre (IPP) in Portugal directly confirms two key points that validate our previous investigation:

Country: The economic data is relevant to Portugal.

Context: The data is from a Portuguese polytechnic institute during a specific time period.

| Metric            | Range (Min to Max) | Confirmed Context                                                                 | Conclusion                                           |
|------------------|-------------------|----------------------------------------------------------------------------------|----------------------------------------------------|
| Unemployment Rate | 7.6% to 16.2%     | Matches the severe economic crisis and subsequent recovery in Portugal (roughly 2008–2018). | Plausible. The highest values are likely accurate historical peaks. |
| Inflation Rate    | -0.8% to 3.7%     | Matches Portugal's largely stable, low-inflation environment during that period, including some deflationary dips. | Plausible. Very stable and reliable data.         |
| GDP               | -4.06 to 3.51     | Matches Portugal's sharp recessionary decline followed by years of recovery growth. | Plausible. Represents real economic volatility.   |


phương pháp xử lý đề xuất (cho mấy cột trên với lại mấy cái course)

***!!!Square Root Transformation!!!***
might or might or have to do this

You only transform the distribution (log/square root) for features that are highly skewed (like GDP and Curricular Units). You must apply scaling to every continuous numeric column to ensure no single variable dominates your model due to its large value range.

***NHẬN XÉT TỔNG QUAN SO FAR***

In [ ]:
df_numeric.columns

In [ ]:
from scipy.stats import norm
import numpy as np

## 2. Phân tích phân phối xác suất

#### Chọn ít nhất hai biến số quan trọng và kiểm tra xem nó có tuân theo phân phối xác suất nào không (chuẩn, Poisson, exponential…) sử dụng biểu đồ phù hợp để minh hoạ.

In [ ]:
col = "Application order"
data = df[col].dropna()  # drop NaN values
mu, std = norm.fit(data)     # fit a normal distribution

plt.figure(figsize=(6,4))
# Histogram
sns.histplot(data, bins=20, kde=False, color='skyblue', stat='density')
# Plot Gaussian
xmin, xmax = data.min(), data.max()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'r', linewidth=2)

plt.title(f'{col} (mu={mu:.2f}, std={std:.2f})')
plt.xlabel(col)
plt.ylabel('Density')
plt.show()

In [ ]:
col = "Admission grade"
data = df_numeric[col].dropna()  # drop NaN values
mu, std = norm.fit(data)     # fit a normal distribution

plt.figure(figsize=(6,4))
# Histogram
sns.histplot(data, bins=20, kde=False, color='skyblue', stat='density')
# Plot Gaussian
xmin, xmax = data.min(), data.max()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'r', linewidth=2)

plt.title(f'{col} (mu={mu:.2f}, std={std:.2f})')
plt.xlabel(col)
plt.ylabel('Density')
plt.show()

In [ ]:
col = "Age at enrollment"
data = df_numeric[col].dropna()  # drop NaN values
mu, std = norm.fit(data)     # fit a normal distribution

plt.figure(figsize=(6,4))
# Histogram
sns.histplot(data, bins=20, kde=False, color='skyblue', stat='density')
# Plot Gaussian
xmin, xmax = data.min(), data.max()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'r', linewidth=2)

plt.title(f'{col} (mu={mu:.2f}, std={std:.2f})')
plt.xlabel(col)
plt.ylabel('Density')
plt.show()

In [ ]:
col = "Curricular units 1st sem (grade)"
data = df_numeric[col].dropna()  # drop NaN values
mu, std = norm.fit(data)     # fit a normal distribution

plt.figure(figsize=(6,4))
# Histogram
sns.histplot(data, bins=20, kde=False, color='skyblue', stat='density')
# Plot Gaussian
xmin, xmax = data.min(), data.max()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'r', linewidth=2)

plt.title(f'{col} (mu={mu:.2f}, std={std:.2f})')
plt.xlabel(col)
plt.ylabel('Density')
plt.show()


In [ ]:
col = "Curricular units 2nd sem (grade)"
data = df_numeric[col].dropna()  # drop NaN values
mu, std = norm.fit(data)     # fit a normal distribution

plt.figure(figsize=(6,4))
# Histogram
sns.histplot(data, bins=20, kde=False, color='skyblue', stat='density')
# Plot Gaussian
xmin, xmax = data.min(), data.max()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'r', linewidth=2)

plt.title(f'{col} (mu={mu:.2f}, std={std:.2f})')
plt.xlabel(col)
plt.ylabel('Density')
plt.show()


## 3. Kiểm định giả thuyết

### a. Tìm hiểu và trình bày các kiểm định thống kê sau: t-test, Chi-square và ANOVA.

### b. Đề xuất hai câu hỏi nghiên cứu có thể kiểm định bằng dữ liệu cho bộ dữ liệu đã chọn

câu hỏi: giới tính khác nhau có ảnh hưởng đến cái dropout rate hay ko

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Tạo bảng tần số (contingency table)
table = pd.crosstab(df['Gender'], df['Target'])
print(table)


In [ ]:
chi2, p, dof, expected = chi2_contingency(table)
print("Chi-square statistic:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies:\n", expected)

Null hypothesis (H0): Giới tính và kết quả học tập không liên quan (độc lập)

Alternative hypothesis (H1): Có sự phụ thuộc giữa giới tính và kết quả học tập

Nếu p < 0.05 → bác bỏ H0 → có sự khác biệt có ý nghĩa

Nếu p >= 0.05 → không bác bỏ H0 → không có bằng chứng về sự khác biệt

Vì p << 0.05, chúng ta bác bỏ H0 → kết luận:

Có sự khác biệt rất rõ ràng về kết quả học tập giữa nam và nữ. Giới tính và Target không độc lập trong dữ liệu này.

In [ ]:

table_norm = table.div(table.sum(axis=1), axis=0)  # chuyển sang tỉ lệ %
table_norm.plot(kind='bar', stacked=True, figsize=(8,5), colormap='Set2')

plt.ylabel("Proportion")
plt.xlabel("Gender (0=Male, 1=Female)")
plt.title("Target distribution by Gender")
plt.show()

In [ ]:
import seaborn as sns

# Bảng tỉ lệ %
table_percent = table.div(table.sum(axis=1), axis=0) * 100

plt.figure(figsize=(6,4))
sns.heatmap(table_percent, annot=True, fmt=".1f", cmap="YlGnBu")
plt.xlabel("Target")
plt.ylabel("Gender (0=Male, 1=Female)")
plt.title("Heatmap of Target distribution by Gender (%)")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(data=df, x='Admission grade', hue='Gender', 
             bins=20, kde=True, palette=['blue','orange'], alpha=0.6, stat='density')

plt.title('Distribution of Admission Grade by Gender')
plt.xlabel('Admission Grade')
plt.ylabel('Density')
plt.legend(title='Gender', labels=['Female','Male'])
plt.show()

wowza! kết quả cho thấy có sự chênh lệch rõ ràng giữa hai giới tính

#### Kiểm định khác biệt độ tuồi giữa các nhóm target bằng anova ###

anova cần mấy biến tuân theo phân phối chuẩn, nên t đi check coi chuẩn ko

In [ ]:
from scipy.stats import shapiro
from scipy.stats import levene

groups = df.groupby('Target')['Age at enrollment']

for name, group_data in groups:
    stat, p = shapiro(group_data.dropna())  # drop NA values
    print(f"{name}: W={stat:.3f}, p={p:.3f}")

groups = [group.dropna() for name, group in df.groupby('Target')['Age at enrollment']]

stat, p = levene(*groups)
print(f"Levene’s test: W={stat:.3f}, p={p:.3f}")


⚠️ This violates the normality assumption for ANOVA.

⚠️ This violates the homogeneity of variance assumption for ANOVA.

Since both normality and equal variance assumptions are violated, one-way ANOVA is NOT appropriate here.

✅ Correct approach:
Use a non-parametric alternative like Kruskal-Wallis test, which does not assume normality or equal variances.

In [ ]:
from scipy.stats import kruskal

dropout_age = df[df['Target']=='Dropout']['Age at enrollment']
enrolled_age = df[df['Target']=='Enrolled']['Age at enrollment']
graduate_age = df[df['Target']=='Graduate']['Age at enrollment']

stat, p = kruskal(dropout_age, enrolled_age, graduate_age)
print(f"Kruskal-Wallis test: H={stat:.3f}, p={p:.3e}")



Since p << 0.05, you reject the null hypothesis. This means there is a statistically significant difference between at least one pair of your groups.

Minh họa

In [ ]:
# Create a DataFrame for plotting
plot_df = df[['Target', 'Age at enrollment']].dropna()

plt.figure(figsize=(8,6))
sns.violinplot(x='Target', y='Age at enrollment', data=plot_df, palette="Set3")
plt.title("Age at Enrollment by Target Group")
plt.ylabel("Age at Enrollment")
plt.xlabel("Target")
plt.show()


Kết luận: Dữ liệu cho thấy mối quan hệ nghịch đảo giữa độ tuổi nhập học và khả năng tốt nghiệp. Về mặt thống kê, những sinh viên nhập học ở độ tuổi truyền thống sớm hơn có khả năng hoàn thành chương trình học cao hơn nhiều so với những sinh viên nhập học muộn hơn.

## 4. Phân tích tương quan giữa các biến

### a. Tìm hiểu và trình bày hệ số tương quan Pearson và Spearman.

Pearson correlation coefficient (PCC) is a correlation coefficient that measures linear correlation between two sets of data. It is the ratio between the covariance of two variables and the product of their standard deviations; thus, it is essentially a normalized measurement of the covariance, such that the result always has a value between −1 and 1. As a simple example, one would expect the age and height of a sample of children from a school to have a Pearson correlation coefficient significantly greater than 0, but less than 1 (as 1 would represent an unrealistically perfect correlation). 

Spearman's rank correlation coefficient or Spearman's ρ is a number ranging from -1 to 1 that indicates how strongly two sets of ranks are correlated. It could be used in a situation where one only has ranked data, such as a tally of gold, silver, and bronze medals. If a statistician wanted to know whether people who are high ranking in sprinting are also high ranking in long-distance running, they would use a Spearman rank correlation coefficient.It assesses how well the relationship between two variables can be described using a monotonic function.

The Spearman correlation between two variables is equal to the Pearson correlation between the rank values of those two variables; while Pearson's correlation assesses linear relationships, Spearman's correlation assesses monotonic relationships (whether linear or not). If there are no repeated data values, a perfect Spearman correlation of +1 or −1 occurs when each of the variables is a perfect monotone function of the other. 

#### b. Áp dụng tính hệ số tương quan Pearson và Spearman giữa các biến số trong bộ dữ liệu đã chọn.

In [ ]:
corr_pearson = df[quantiative_columns].corr(method="pearson")
corr_spearman = df[quantiative_columns].corr(method="spearman")

#### c. Sử dụng biểu đồ phù hợp để trực quan hóa mối quan hệ giữa các biến. (Heatmap)

In [ ]:
# Heatmap for Pearson
plt.figure(figsize=(10, 8))
sns.heatmap(corr_pearson, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Pearson Correlation Matrix")
plt.show()

# Heatmap for Spearman
plt.figure(figsize=(10, 8))
sns.heatmap(corr_pearson, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Spearman Correlation Matrix")
plt.show()

#### d. Đưa ra nhận xét về ý nghĩa thực tiễn của các mối tương quan.

- **Tính kế thừa và dự báo mạnh mẽ (Quan trọng nhất):** Tương quan giữa Học kỳ 1 và Học kỳ 2 là rất cao (màu đỏ đậm, hệ số > 0.9). Kết quả học tập kỳ đầu là chỉ báo dự báo chính xác nhất cho kỳ sau. Sinh viên có xu hướng duy trì phong độ ổn định; ai học tốt kỳ 1 thường sẽ học tốt kỳ 2 và ngược lại. Nhà trường có thể dùng kết quả kỳ 1 để sớm phát hiện sinh viên cần hỗ trợ.
- **Mối quan hệ giữa Nỗ lực và Kết quả:** Có sự tương quan thuận chặt chẽ giữa số môn đăng ký/tham gia đánh giá (enrolled/evaluations) và số môn đạt (approved/credited). Mức độ tích cực tham gia học tập và thi cử quyết định lớn đến kết quả đầu ra. Tuy nhiên, hệ số không phải tuyệt đối (1.0), cho thấy việc "chăm đi thi" là điều kiện cần nhưng chưa đủ, vẫn tồn tại tỷ lệ rớt môn nhất định.
- **Yếu tố Kinh tế vĩ mô không tác động trực tiếp:** Các chỉ số GDP, Lạm phát, Thất nghiệp có hệ số tương quan gần bằng 0 (màu xanh nhạt/trắng) với các biến kết quả học tập.Trong phạm vi dữ liệu này, bối cảnh kinh tế bên ngoài không ảnh hưởng đáng kể đến thành tích học tập của cá nhân sinh viên. Việc học kém hay giỏi phụ thuộc vào yếu tố nội tại hơn là môi trường kinh tế vĩ mô.
- **Tuổi tác không phải là rào cản:** Biến "Age at enrollment" có tương quan rất thấp với các kết quả học tập. Độ tuổi nhập học (già hay trẻ) không quyết định thành công hay thất bại trong môi trường đại học.

***Tóm lại:*** Để cải thiện chất lượng đào tạo, nên tập trung giám sát kết quả ngay từ Học kỳ 1 và khuyến khích sinh viên tham gia đầy đủ các bài đánh giá, thay vì lo ngại về các yếu tố nhân khẩu học (tuổi) hay kinh tế bên ngoài.

## 5. Hồi quy đa biến (Multiple Linear Regression)

### a. Tìm hiểu và trình bày lý thuyết về Hồi quy đa biến.

Hồi quy đa biến là một kĩ thuật thống kê sử dụng nhiều biến độc lập (các thuộc tính) để dự đoán kết quả của một biến phụ thuốc, các biến độc lập này lý tưởng sẽ có ý nghĩa để giải thích sẽ mô hình huấn luyện này. Mục đích của hồi quy đa biến là mô hình hóa mối tương quan tuyến tính giữa các biến độc lập với biến phụ thuộc. Về cơ bản, hồi quy đa biến là một hồi quy tuyến tính vì nó bao gồm nhiều hơn một biến độc lập.

### b. Xây dựng và đánh giá mô hình hồi quy tuyến tính đa biến nhằm dự đoán một biến phụ thuộc (biến mục tiêu) dựa trên nhiều biến độc lập trong bộ dữ liệu đã chọn.

Ở đây ta định nghĩa rằng biến phụ thuộc là Curricular units 2nd sem (grade) (điểm bình quân học kỳ 2 từ 0 tới 20) và các biến còn lại là biến phụ thuộc để minh họa cho hồi quy đa biến do Target của tập dữ liệu là biến định tính (nghỉ học, đang học, tốt nghiệp).

Chúng tôi đã chủ động tránh việc áp dụng One-Hot Encoding toàn phần cho các đặc trưng có tính đa dạng cao (high-cardinality) – như Nghề nghiệp và Khóa học – để ngăn chặn 'Lời nguyền của số chiều' (The Curse of Dimensionality).

Rất nhiều giá trị phân loại trong số này có độ hỗ trợ (Support) thấp (chiếm dưới 1% tập dữ liệu), tạo ra các 'Đặc trưng thưa' (Sparse Features) dẫn đến việc mô hình mất ổn định và bị quá khớp (overfitting).

Thay vào đó, chúng tôi ưu tiên các đặc trưng có độ hỗ trợ cao, bao quát được toàn bộ tổng thể. Điều này đảm bảo mô hình sẽ học được các quy luật hành vi tổng quát thay vì chỉ 'học vẹt' (ghi nhớ) các nhóm nhỏ lẻ.

Support là tỉ lệ giữa độ tần suất của một biến với cả tập dữ liệu. Nó được sử dụng để tìm một tập các biến xuất hiện có thường xuyên hay không trong tập dữ liệu. Công thức:
$$ \text{Support}(X) = \frac{\text{Number of transactions containing itemset X}}{\text{Total number of transactions}} $$
Sau đây chúng tôi sẽ thử nghiệm và áp dụng công thức này lên tất cả các cột trong tập dữ liệu để xét xem thử mức độ thưa thớt của các cột trên tập dữ liệu 

In [ ]:
# 1. Define the Continuous Columns
continuous_cols = [
    'Curricular units 1st sem (grade)', 
    'Curricular units 2nd sem (grade)',
    'Curricular units 1st sem (enrolled)', 
    'Curricular units 2nd sem (enrolled)',
    'Curricular units 1st sem (evaluations)', 
    'Curricular units 2nd sem (evaluations)',
    'Curricular units 1st sem (approved)', 
    'Curricular units 2nd sem (approved)',
    'Curricular units 1st sem (credited)', 
    'Curricular units 2nd sem (credited)',
    'Curricular units 1st sem (without evaluations)', 
    'Curricular units 2nd sem (without evaluations)',
    'Age at enrollment', 
    'Unemployment rate', 
    'Inflation rate', 
    'GDP',
    'Application order',
    'Previous qualification (grade)',
    'Admission grade'
]

# 2. Dynamically identify ALL other columns (The "Hidden" Categories)
df.columns = df.columns.str.strip() 
categorical_candidates = [col for col in df.columns if col not in continuous_cols]

# 3. Run the Audit
audit_data = []

for col in categorical_candidates:
    # Get the distribution percentages
    counts = df[col].value_counts(normalize=True) * 100
    
    total_categories = len(counts)
    rare_categories = len(counts[counts < 1.0]) 
    
    if total_categories > 0:
        sparsity_rate = (rare_categories / total_categories) * 100 # Support value
    else:
        sparsity_rate = 0
    
    audit_data.append({
        'Feature': col,
        'Total Cats': total_categories,
        'Rare Cats (<1%)': rare_categories,
        'Sparsity Rate (%)': round(sparsity_rate, 1)
    })

# 4. Sort and Display
audit_df = pd.DataFrame(audit_data).sort_values(by='Sparsity Rate (%)', ascending=False)

# FIX: Explicitly remove any lingering tabs in the Feature column before plotting
audit_df['Feature'] = audit_df['Feature'].astype(str).str.replace('\t', '', regex=False)

print(f"--- FULL DATASET AUDIT ({len(categorical_candidates)} Columns Scanned) ---")
print(audit_df)

# 5. Visualization
plt.figure(figsize=(12, 8))

sns.barplot(
    x='Sparsity Rate (%)', 
    y='Feature', 
    data=audit_df, 
    palette='magma', 
    hue='Feature', 
    legend=False
)

plt.title('Sparsity Audit: Which columns are mostly "Junk"?', fontsize=14, fontweight='bold')

# FIX: Removed plt.legend() because it was empty and causing a warning
plt.tight_layout()
plt.show()

- **Lý do ưu tiên các Đặc trưng có "Độ thưa thấp" (0%)**
    - Quá trình kiểm tra cho thấy các đặc trưng có số lượng phân loại lớn (high-cardinality) như Nationality (Quốc tịch) và Occupation (Nghề nghiệp) bị thưa thớt nghiêm trọng, trong đó hơn 80% các danh mục xuất hiện trong dưới 1% tập dữ liệu. Việc đưa các đặc trưng này vào sẽ tạo ra hai vấn đề nghiêm trọng:
    - Quá khớp (Overfitting): Mô hình sẽ "ghi nhớ" các danh mục hiếm (ví dụ: một công việc cụ thể chỉ có 3 sinh viên làm) thay vì học các mẫu tổng quát hóa.
    - Không ổn định (Instability): Các hệ số được suy ra từ các mẫu có kích thước quá nhỏ sẽ không đáng tin cậy về mặt thống kê.

Vì tập dữ liệu này đã lớn sẵn và khi được phân rã các biến định tính bằng one hot encoding sẽ làm cho tập dữ liệu càng rối thêm. Để làm cho mô hình đơn giản, dễ hiểu và không quá dài dòng thì chúng tôi giới hạn danh sách ứng viên vào các đặc trưng có độ thưa 0% (Độ hỗ trợ cao/High Support), chúng tôi đảm bảo rằng mọi biến đều áp dụng cho một phần lớn của tập hợp sinh viên. Điều này đảm bảo mô hình học được các mẫu hành vi toàn cục thay vì nhiễu cục bộ.

- **Loại trừ Biến "Target" (Rò rỉ dữ liệu)**: Mặc dù biến Target (Tốt nghiệp/Bỏ học) xuất hiện trong danh sách Độ thưa 0%, nó bị loại trừ khỏi danh sách ứng viên do vấn đề Rò rỉ dữ liệu (Data Leakage). Target đại diện cho kết quả cuối cùng của văn bằng (thông tin tương lai). Trong kịch bản dự đoán thực tế (ví dụ: dự đoán điểm số khi bắt đầu Học kỳ 2), chúng ta không thể biết liệu sinh viên đó có tốt nghiệp nhiều năm sau hay không. Việc sử dụng Target để dự đoán Điểm học kỳ 2 sẽ thổi phồng độ chính xác một cách nhân tạo bằng thông tin không có sẵn tại thời điểm dự đoán.

- **Về GDP**: Chúng tôi giữ lại GDP (và các chỉ số kinh tế liên quan) làm ứng viên. Khác với Target, các giá trị này thường được ghi nhận tại thời điểm nhập học hoặc là các trung bình kinh tế vĩ mô hàng năm đã biết trong suốt học kỳ. Chúng cung cấp bối cảnh môi trường hợp lệ mà không vi phạm logic thời gian.

- **Danh sách Ứng viên Sơ bộ (Trước khi Kiểm định Thống kê)**: Dựa trên tiêu chí Độ thưa 0%, chúng tôi giữ lại toàn bộ danh sách các biến an toàn (High Support) dưới đây để tiến hành các bước kiểm định tiếp theo:
    - Nhóm Tài chính: Tuition fees up to date, Debtor, Scholarship holder.
    - Nhóm Nhân khẩu học: Age at enrollment, Gender, Displaced, International.
    - Nhóm Hành vi/Khác: Daytime/evening attendance, Educational special needs.
    - Nhóm Kinh tế/Học thuật: GDP, Curricular units 1st sem (grade).

- **Kế hoạch Kiểm định Thống kê (Validation Plan)**: Để gạn lọc danh sách sơ bộ này, chúng tôi sẽ áp dụng hai bước kiểm tra thống kê cho tất cả các ứng viên trong danh sách trên:
    - Bước 1: Kiểm tra VIF để Phát hiện các biến bị dư thừa (Multicollinearity). Chúng tôi sẽ kiểm tra ván đề đa cộng tuyến của các ứng cử viên này và loại bỏ (nếu có) các thuộc với giá trị VIF cao (lớn hơn 5).
    - Bước 2: Chạy mô hình OLS với tất cả ứng viên còn lại. Nếu một biến có P-value > 0.05, nó chứng tỏ biến đó không đóng góp thông tin giá trị khi đã có các biến khác, và sẽ bị loại bỏ.

Phát hiện đa cộng tuyến trước khi áp dụng các biến vào mô hình hồi quy và loại bỏ các ứng cử viên được xem là không quan trọng về mặt thống kê

In [ ]:
# Step 1
# We will check vif to avoid multicolinearity
# 1. Define list of 0% sparsity features
redundancy_columns_check = [
    'Daytime/evening attendance',
    'Displaced',
    'Educational special needs',
    'Debtor', 
    'Tuition fees up to date',
    'Gender',
    'Scholarship holder',
    'International',
    'Age at enrollment',
]

redundancy_df_check = df[redundancy_columns_check]
# 2. Add a constant (intercept) for the VIF calculation
# (statsmodels OLS requires an intercept to calculate VIF correctly)
X_vif = redundancy_df_check.assign(const=1)

# 3. Calculate VIF
vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) 
                   for i in range(len(X_vif.columns))]

# 4. Sort and Display
print(vif_data.sort_values(by="VIF", ascending=False))

Kết quả cho thấy tất cả thuộc tính đều có giá trị vif rất thấp nên tạm thời có thể đưa vô mô hình ols một cách an toàn 

In [ ]:
# Step 2
# We create a model with all these candidates
X_challenger = sm.add_constant(redundancy_df_check)
y = df['Curricular units 2nd sem (grade)']

model_challenger = sm.OLS(y, X_challenger).fit()

print("--- EVIDENCE B: The 'Challenger' Model Results ---")
print(model_challenger.summary())

Chúng tôi sẽ tập trung vào cột P|t|, nếu giá trị hơn 0.05 thì biến sẽ không đáng kể về mặt thống kê và sẽ được loại bỏ ra khỏi mô hình hồi quy đa biến OLS.

Candidates for Removal (The "Cleanup List"):

Daytime/evening attendance (0.446) → Drop.

International (0.384) → Drop.

Displaced (0.361) → Drop.

Debtor (0.277) → Drop.

Educational special needs (0.087) → Drop

Sau đó chúng tôi sẽ huấn luyện lại mô hình với các biến có ý nghĩa thống kê

In [ ]:
# 1. The Evidence-Based Feature List
# - Low Sparsity (Stable)
# - No Leakage (Realistic)
# - Statistically Significant (Useful)
features = [
    'Curricular units 1st sem (grade)',  # Academic Momentum
    'Tuition fees up to date',           # Financial Commitment
    'Scholarship holder',                # Financial Incentive
    'Age at enrollment',                 # Demographic Risk
    'GDP',                               # Economic Context (At Enrollment)
    'Gender',                            # Demographic Baseline
]

# 2. Prepare Data
X_final = df[features].copy()
X_final = sm.add_constant(X_final)
y = df['Curricular units 2nd sem (grade)']

# 3. Run OLS
model_final = sm.OLS(y, X_final).fit()

# 4. Show the World
print(model_final.summary())

- **Độ phù hợp của mô hình (Model Fit)**: Hệ số xác định R2=0.712 cho thấy mô hình giải thích được 71.2% sự biến thiên của biến phụ thuộc (Điểm môn học kỳ 2). Đây là mức cao đối với dữ liệu xã hội/giáo dục. Chỉ số Prob (F-statistic) xác nhận mô hình tổng thể là tin cậy, các biến độc lập thực sự có tác động đến biến phụ thuộc.

- **Đánh giá các biến độc lập (Coefficients)**: Tất cả 6 biến đưa vào mô hình đều có ý nghĩa thống kê (P>∣t∣ đều nhỏ hơn 0.05). 
    - Các biến có tác động tích cực (Làm tăng điểm):
        - Điểm kỳ 1 (Curricular units 1st sem): Là biến quan trọng nhất (t-stat cao nhất). Cứ tăng 1 điểm kỳ 1 thì điểm kỳ 2 tăng trung bình 0.86 điểm. 
        - Đóng học phí (Tuition fees up to date), sinh viên đóng học phí đúng hạn có điểm cao hơn đáng kể (hệ số 1.37).
        - Học bổng & GDP: Có tác động dương nhưng mức độ nhẹ hơn.
    - Các biến tác động tiêu cực (Làm giảm điểm):
        - Hệ số giới tính (Gender) -0.33 cho thấy có sự chênh lệch điểm số giữa các giới tính (nhóm được mã hóa là 1 có điểm thấp hơn nhóm 0).
        - Tuổi (Age at enrollment), sinh viên nhập học càng lớn tuổi, điểm càng có xu hướng giảm nhẹ (hệ số -0.015).

- **Kiểm định giả thiết (Diagnostics)**: Không có tự tương quan, giá trị Durbin-Watson 2.012 (xấp xỉ 2) là lý tưởng. Đa cộng tuyến thấp: Cond. No.: 147 nằm trong ngưỡng an toàn, không có hiện tượng đa cộng tuyến nghiêm trọng. Vấn đề phân phối chuẩn: Các chỉ số Omnibus, Jarque-Bera, Kurtosis rất cao cho thấy phần dư (residuals) không phân phối chuẩn. Tuy nhiên, với kích thước mẫu lớn (N=4424), điều này không quá ảnh hưởng đến tính vững của các hệ số hồi quy, nhưng cần lưu ý khi xét khoảng tin cậy.

**Kết luận**: Mô hình mạnh và đáng tin cậy để dự báo điểm kỳ 2, trong đó kết quả học tập kỳ 1 là yếu tố dự báo then chốt nhất.